# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Последовательность выполнения шагов**
* Загрузка файлов, их изучение и подготовка к дальнейшему анализу
* Предобработка данных (обработка пропусков, дубликатов, аномалий)
* Разделение клиентов по их уровню дохода
* Выделение основных целей кредитов с помощью лемматизации
* Ответы на вопросы исследования:
    1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
    2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
    3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    4. Как разные цели кредита влияют на его возврат в срок?
* Выводы по исследованию

## Загрузка и обзор файлов из данных

In [2]:
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
 
df = pd.read_csv('/datasets/data.csv') 
df.head(5) #посмотирм первые 5 строк датасета

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Сразу можем заметить, что имеются артефакты в столбце `days_employed` - значения трудового стажа не может быть отрицательным. 

Кроме того, видно наличие верхнего и нижнего регистра в столбце `education`. 

Пол заёмщиков указан на английском языке, что не соответствует общему стилю таблице на русском.

В дальнейшем это исправим.

In [4]:
df.info() #получаем общую информации о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
df.columns #посмотрим названия колонок

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

В таблице находятся 12 столбцов. 

Имеется 2 колонки типа `float64`, 5 колонок типа `int64` и 5 колонок типа `object`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита


**Вывод**

Количество значений в столбцах различаются, что означает наличие пропусков в данных.

Написание столбцов соответствуют регистру Snake case.

Для дальнейшей работы с данными необходимо устранить проблемы в данных.

## Предобработка данных

### Стиль заголовков

Для удобства переименуем столбец `dob_years` на `deb_years`. Сокращение ведь от слова debtor, так будет правильней. 

In [6]:
df = df.rename(columns={'dob_years':'deb_years'}) #переименование столбца
df.columns #проверка названий столбцов

Index(['children', 'days_employed', 'deb_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Вывод**

Название столбца изменили, всё работает.

### Обработка пропусков

In [7]:
df.isna().sum() # посмотрим количество пропусков по столбцам

children               0
days_employed       2174
deb_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [8]:
display(df[df['days_employed'].isna()].head(5)) # посмотрим какие именно пропуски присутствуют в колонке days_employed

,children,days_employed,deb_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


**Вывод**

Можно заметить, что и в столбце `days_employed`, и в столбце `total_income` присутствуют пропущенные значения типа NaN. Причём каждому пропуску в одном из этих столбцов соответствует пропуск в другом.

Причиной их появления мог быть сбой в системе сбора данных заёмщиков.

Пропуски присутствуют в таких колонках, как `days_employed` и `total_income`. Если удалить строки, в которых присутствуют эти пропуски, то это может сказаться на всём исследовании. Будем заменять пропуски медианным значениями по этим столбцам. Заменяем именно медианным значениями, так как в столбцах присутствуют значения, которые сильно выделяются среди других.

Создадим переменную, в которой запишем медианное значение по столбцу `days_employed`, всплеск медианных значений в этом случае не является критическим, так как этот столбец не является важным для дальнейшего исследования. Затем присвоим эти значения строкам с пропусками. 

Для работы с пропусками столбца `total_income` создадим 9 переменных, в которых будут медианные значения по этому столбцу в зависимости от условий, где-то это будет условие по типу образования и занятости, а где-то только по образованию. После этого создадим функцию, которая будет возвращать значения наших раннее созданных переменных в зависимости от образования и типа занятости, если значение в столбце отсутствует, и возвращать само значение, если оно присутствует. Затем используем функцию fillna(), где её аргументом будет ранее созданная нами функция.

In [9]:
df['education'] = df['education'].str.lower() #сразу приведём столбец education к нижнему регистру для будущей функции
display(df[df['total_income'].isna()]['education'].value_counts()) #посмотрим, для каких значений образования и
df[(df['total_income'].isna())]['income_type'].value_counts() #типа занятости будем создавать переменные медианных значений

среднее                1540
высшее                  544
неоконченное высшее      69
начальное                21
Name: education, dtype: int64

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [10]:
days_employed_median = df['days_employed'].median() #переменная для столбца days_employed

#медиана зарплаты пенсионеров со средним образованием
pensioner_education_middle_median = df[(df['income_type'] == 'пенсионер') & 
                                       (df['education'] == 'среднее')]['total_income'].median() 

#медиана зарплаты пенсионеров с высшим образованием
pensioner_education_uni_median = df[(df['income_type'] == 'пенсионер') &
                                    (df['education'] == 'высшее')]['total_income'].median()

#сотрудник и госслужащий похожи по значениям зарплаты, поэтому их можно объединить в одну переменную
#в данном случае условием будет среднее образование
employee_education_middle_median = df[((df['income_type'] == 'сотрудник') | (df['income_type'] == 'госслужащий')) &
                                      (df['education'] == 'среднее')]['total_income'].median()

#то же самое, что и выше, только для высшего образования
employee_education_uni_median = df[((df['income_type'] == 'сотрудник') | (df['income_type'] == 'госслужащий')) &
                                    (df['education'] == 'высшее')]['total_income'].median()

#берём отдельно медианное значение для предпринимателя, так как значения зарплаты сильно отличаются от других типов занятости
businessman_median = df[(df['income_type'] == 'предприниматель')]['total_income'].median()

#медиана зарплаты с типом занятости - компаньон, образование среднее
companion_education_middle_median = df[(df['income_type'] == 'компаньон') &
                                       (df['education'] == 'среднее')]['total_income'].median()

#что и выше, только образование высшее
companion_education_uni_median = df[(df['income_type'] == 'компаньон') &
                                    (df['education'] == 'высшее')]['total_income'].median()

#так как все группы с начальным образованием имеют приблизительно равные значения зарплаты, то
#возьмём медианные значения по всем типам занятости с начальным образованием, не разбивая их по типам занятости
all_education_elementary = df[df['education'] == 'начальное']['total_income'].median() 

#что и выше, только для неоконченного высшего образования
all_education_not_finished = df[df['education'] == 'неоконченное высшее']['total_income'].median() 


def na_replace_median(row): #функция для замены пропущенных значений столбца total_income
    
    total = row['total_income']
    typpe = row['income_type']
    edu = row['education']
    
    if total > 0: #если значение зарплаты присутствует, то возвращаем само значение
        return(total)
    else: #иначе обрабатываем пропуски с учётом наших логических условий и возвразем соответствующим медианы зарплаты
        if typpe == 'пенсионер':
            if edu == 'среднее':
                return(pensioner_education_middle_median)
            elif edu == 'высшее':
                return(pensioner_education_uni_median)
            
        elif typpe == 'сотрудник' or typpe =='госслужащий':
            if edu == 'среднее':
                return(employee_education_middle_median)
            elif edu == 'высшее':
                return(employee_education_uni_median)
        
        elif typpe == 'предприниматель':
            return(businessman_median)
        
        elif typpe == 'компаньон':
            if edu == 'среднее':
                return(companion_education_middle_median )
            elif edu == 'высшее':
                return(companion_education_uni_median)
                       
        if edu == 'начальное':
            return(all_education_elementary)
        elif edu == 'неоконченное высшее':
            return(all_education_not_finished)           
    
    
df['days_employed'] = df['days_employed'].fillna(days_employed_median) # заменяем пустые значениями переменных
df['total_income'] = df.apply(na_replace_median, axis=1) #применяем нашу функцию на датафрейм
display(df['total_income'].value_counts().head(10)) #проверим, как сработала наша функция
df.isna().sum() # проверим количество пропусков

136585.186586    898
114842.854099    345
159070.690289    300
167170.112319    299
201785.400018    188
160115.398644     70
144240.768611     57
117137.352825     22
499163.144947      2
126589.160240      1
Name: total_income, dtype: int64

children            0
days_employed       0
deb_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Таким образом, мы разорбрались со всеми пропусками в данных, заменив их медианными значениями по столбцам.

### Обработка артефактов

In [16]:
df['days_employed'] = df['days_employed'].abs() #применим функцию модуля к столбцу days_employed, чтобы поменять знак у отрицательных значений
df.head()
df['children'] = df['children'].abs() #функция модуля для столбца children
df['children'] = df['children'].replace (20, 2) #изменение значения с 20 на 2
df['deb_years'] = df['deb_years'].replace (0, 18) #изменение значения с 0 на 18
df['days_employed'].value_counts() #проверка столбца days_employed

1203.369529    2175
986.927316        1
1893.222792       1
4236.274243       1
6620.396473       1
               ... 
2849.351119       1
5619.328204       1
448.829898        1
1687.038672       1
582.538413        1
Name: days_employed, Length: 19351, dtype: int64

### Замена типа данных

In [17]:
df['days_employed'] = df['days_employed'].astype('int64') # заменям тип данных в столбце days_employed на int64
df['total_income'] = df['total_income'].astype('int64') # в столбце total_income
df.info() # проверка типов данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
deb_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Мы заменили тип данных в двух столбцах методом `astype()`. Использовали мы его так как он позволяет перевести тип чисел с плавающей точкой в целочисленный тип, чего не позволяет делать метод `to_numeric`.

Таким образом, в нашей таблице остались только два типа данных: `int64` и `object`

### Обработка дубликатов

In [18]:
display(df.duplicated().sum()) #посмотрим количество дубликатов в датафрейме
display(df.duplicated().sum()) #проверим, не изменилось ли количество дубликатов
df = df.drop_duplicates().reset_index(drop=True)  #удалим дубликаты из таблицы
display(df.duplicated().sum()) #снова проверяем на наличие дубликатов

71

71

0

**Вывод**

Мы посмотрели количество дубликатов в таблице, привели столбец `education` к нижнему регистру, а затем успешно от избавились от дубликатов методом `drop_duplicates()`. 

Причиной появления дубликатов мог быть сбой в системе сбора данных заёмщиков.

### Лемматизация

In [19]:
from pymystem3 import Mystem #импорт инструмента для лемматизации
m = Mystem() 

df['lemmas'] = df['purpose'].apply(m.lemmatize) #применение функции лемматизации для столбца 'purpose'
df['lemmas'].value_counts().head() #посмотрим результат работы функции

[автомобиль, \n]                         972
[свадьба, \n]                            791
[на,  , проведение,  , свадьба, \n]      768
[сыграть,  , свадьба, \n]                765
[операция,  , с,  , недвижимость, \n]    675
Name: lemmas, dtype: int64

**Вывод**

Мы лемматизировали строки столбца `purpose` и сохранили результат в новом столбце `lemmas`. 

Можно сразу увидеть, что основными целями кредита являются покупка автомобиля, свадьба, недвижимость и образование.

В дальнейшем используем этот столбец для категоризации данных в зависимости от целей кредита.

### Категоризация данных

In [20]:
children_df = df[['children', 'debt']] #наш новый датафрейм для категоризации по количеству детей

def child_group(child): #создаём функцию для категоризации по количеству детей
    
    if child == 0:
        return 'нет детей'
    elif child == 1:
        return 'один ребёнок'
    elif child > 1:
        return 'больше одного ребёнка'

children_df['children_group'] = children_df['children'].apply(child_group) #применяем функцию на датафрейм 
children_df.head() #проверяем результат

,children,debt,children_group
0,1,0,один ребёнок
1,1,0,один ребёнок
2,0,0,нет детей
3,3,0,больше одного ребёнка
4,0,0,нет детей


In [21]:
income_df = df[['total_income', 'debt']] #новый датафрейм для категоризации по доходу

#для случая дохода возьмём интервал медианных значений с верхним и нижним пределом, которые сохраним в двух переменных 
income_median_bottom = (df['total_income'].median() * 0.825) #переменная, в которой сохраним верхний предел 
income_median_top = (df['total_income'].median() * 1.175) #нижний

def income_group(income): #функция для категоризации по доходу
    
    if income <= income_median_bottom:
        return 'доход ниже среднего'
    elif (income_median_bottom < income < income_median_top):
        return 'средний доход'
    elif income >= income_median_top:
        return 'высокий доход'
    
income_df['income_group'] = income_df['total_income'].apply(income_group) #применяем функцию на датафрейм     
income_df.head() #проверяем результат

,total_income,debt,income_group
0,253875,0,высокий доход
1,112080,0,доход ниже среднего
2,145885,0,средний доход
3,267628,0,высокий доход
4,158616,0,средний доход


In [24]:
purpose_df = df[['lemmas', 'debt']] #новый датафрейм для категоризации по целям кредита

def purpose_group(purpose): #функция для категоризации по целям кредита
    
    if 'автомобиль' in purpose:
        return('автомобиль')
    elif 'свадьба' in purpose:
        return('свадьба')
    elif 'образование' in purpose:
        return('образование')
    else:
        return('недвижимость')
        
purpose_df['purpose_group'] = purpose_df['lemmas'].apply(purpose_group) #применяем функцию на датафрейм    
purpose_df.head() #проверяем результат

,lemmas,debt,purpose_group
0,"[покупка, , жилье, \n]",0,недвижимость
1,"[приобретение, , автомобиль, \n]",0,автомобиль
2,"[покупка, , жилье, \n]",0,недвижимость
3,"[дополнительный, , образование, \n]",0,образование
4,"[сыграть, , свадьба, \n]",0,свадьба


**Вывод**

Мы категоризировали данные по определённым столбцам трёх новых датафреймов с помощью трёх функций. Получившийся результат добавили в наши датафреймы в виде новых столбцов. Теперь мы можем наглядно посмотреть к какой группе в зависимости от дохода, семейного положения или наличия детей принадлежит клиент банка

## Исследовательский анализ данных

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
#с помощью трёх переменных посмотрим процент должников в зависимости от количества детей
#в этих переменных введём условие, чтобы у клиента банка был долг и чтобы он принадлежал к одной из категоризированных групп
#найдя количество должников, разделим его на общее количество представителей группы и умножим на 100 для получения процентного значения
#затем округлим получившееся значения до одного знака после запятой

child_0 = children_df[(children_df['debt'] == 1) & 
                      (children_df['children_group'] == 'нет детей')]['debt'].count() / (children_df[children_df['children_group'] == 'нет детей']['debt'].count()) * 100
child_0 = child_0.round(1)
print(f'процент должников без детей: {child_0}%')

child_1 = children_df[(children_df['debt'] == 1) &
                      (children_df['children_group'] == 'один ребёнок')]['debt'].count() / (children_df[children_df['children_group'] == 'один ребёнок']['debt'].count()) * 100
child_1 = child_1.round(1)
print(f'процент должников с одним ребёнком: {child_1}%')

child_2 = children_df[(children_df['debt'] == 1) &
                      (children_df['children_group'] == 'больше одного ребёнка')]['debt'].count() / (children_df[children_df['children_group'] == 'больше одного ребёнка']['debt'].count()) * 100
child_2 = child_2.round(1)
print(f'процент должников, где больше одного ребёнка: {child_2}%')

процент должников без детей: 7.5%
процент должников с одним ребёнком: 9.2%
процент должников, где больше одного ребёнка: 9.3%


**Вывод**

Можем сразу заметить, что процент должников по кредиту увеличивается сразу, как в семье появляется хотя бы один ребёнок.

Это неудивительно, так как доход начинает уходить на ребёнка, а на возвращение кредита в срок дохода может уже не хватить.

При этом процент должников с повышением количества детей незначительно увеличивается.

Таким образом, наличие детей негативно влияет на возврат кредита в срок.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
#Аналогично предыдущему случаю посмотрим процент должников в зависимости от семейного положения

family_0 = df[(df['debt'] == 1) & (df['family_status'] == 'женат / замужем')]['debt'].count() / (df[df['family_status'] == 'женат / замужем']['debt'].count()) * 100
family_0 = family_0.round(1)
print(f'процент должников, находящихся в браке: {family_0}%')

family_1 = df[(df['debt'] == 1) & (df['family_status'] == 'гражданский брак')]['debt'].count() / (df[df['family_status'] == 'гражданский брак']['debt'].count()) * 100
family_1 = family_1.round(1)
print(f'процент должников, находящихся в гражданском браке : {family_1}%')

family_2 = df[(df['debt'] == 1) & (df['family_status'] == 'Не женат / не замужем')]['debt'].count() / (df[df['family_status'] == 'Не женат / не замужем']['debt'].count()) * 100
family_2 = family_2.round(1)
print(f'процент должников, не находящихся в браке: {family_2}%')

family_3 = df[(df['debt'] == 1) & (df['family_status'] == 'в разводе')]['debt'].count() / (df[df['family_status'] == 'в разводе']['debt'].count()) * 100
family_3 = family_3.round(1)
print(f'процент должников, находящихся в разводе: {family_3}%')

family_4 = df[(df['debt'] == 1) & (df['family_status'] == 'вдовец / вдова')]['debt'].count() / (df[df['family_status'] == 'вдовец / вдова']['debt'].count()) * 100
family_4 = family_4.round(1)
print(f'процент должников, потерявших супруга(супругу): {family_4}%')

процент должников, находящихся в браке: 7.5%
процент должников, находящихся в гражданском браке : 9.3%
процент должников, не находящихся в браке: 9.8%
процент должников, находящихся в разводе: 7.1%
процент должников, потерявших супруга(супругу): 6.6%


**Вывод**

Здесь мы можем заметить, что самый большой процент должников у группы не находящихся в браке и у группы находящихся в гражданском браке. Самый низкий у группы утерявших партнёра.

Семейное положение влияет на возврат кредита в срок, и выше мы можем это наглядно увидеть.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
#Аналогично случаю выше посмотрим процент должников в зависимости от дохода

income_0 = income_df[(income_df['debt'] == 1) &
                     (income_df['income_group'] == 'доход ниже среднего')]['debt'].count() / (income_df[income_df['income_group'] == 'доход ниже среднего']['debt'].count()) * 100
income_0 = (income_0).round(1)
print(f'процент должников с доходом ниже среднего: {income_0}%')

income_1 = income_df[(income_df['debt'] == 1) &
                     (income_df['income_group'] == 'средний доход')]['debt'].count() / (income_df[income_df['income_group'] == 'средний доход']['debt'].count()) * 100
income_1 = income_1.round(1)
print(f'процент должников со средним доходом: {income_1}%')

income_2 = income_df[(income_df['debt'] == 1) &
                     (income_df['income_group'] == 'высокий доход')]['debt'].count() / (income_df[income_df['income_group'] == 'высокий доход']['debt'].count()) * 100
income_2 = income_2.round(1)
print(f'процент должников доходом выше среднего: {income_2}%')

процент должников с доходом ниже среднего: 8.2%
процент должников со средним доходом: 8.7%
процент должников доходом выше среднего: 7.5%


**Вывод**

Здесь мы можем наблюдать, что наибольший процент должников находится в группе со средним доходом. А наименьший в группе с доходом выше среднего.

Тогда уровень дохода так же влияет на возврат кредита в срок. 

### Как разные цели кредита влияют на его возврат в срок?

In [39]:
#Аналогично случаю выше посмотрим процент должников в зависимости от цели кредита
purpose_0 = purpose_df[(purpose_df['debt'] == 1) &
                       (purpose_df['purpose_group'] == 'недвижимость')]['debt'].count() / (purpose_df[purpose_df['purpose_group'] == 'недвижимость']['debt'].count()) * 100
purpose_0 = purpose_0.round(1)
print(f'процент должников, которые берут кредит для для недвижимости: {purpose_0}%')

purpose_1 = purpose_df[(purpose_df['debt'] == 1) &
                       (purpose_df['purpose_group'] == 'автомобиль')]['debt'].count() / (purpose_df[purpose_df['purpose_group'] == 'автомобиль']['debt'].count()) * 100
purpose_1 = purpose_1.round(1)
print(f'процент должников, которые берут кредит для для автомобиля: {purpose_1}%')

purpose_2 = purpose_df[(purpose_df['debt'] == 1) &
                       (purpose_df['purpose_group'] == 'образование')]['debt'].count() / (purpose_df[purpose_df['purpose_group'] == 'образование']['debt'].count()) * 100
purpose_2 = purpose_2.round(1)
print(f'процент должников, которые берут кредит для для образования: {purpose_2}%')

purpose_3 = purpose_df[(purpose_df['debt'] == 1) &
                       (purpose_df['purpose_group'] == 'свадьба')]['debt'].count() / (purpose_df[purpose_df['purpose_group'] == 'свадьба']['debt'].count()) * 100
purpose_3 = purpose_3.round(1)
print(f'процент должников, которые берут кредит для для свадьбы: {purpose_3}%')

процент должников, которые берут кредит для для недвижимости: 7.2%
процент должников, которые берут кредит для для автомобиля: 9.4%
процент должников, которые берут кредит для для образования: 9.2%
процент должников, которые берут кредит для для свадьбы: 8.0%


**Вывод**

Здесь мы можем видеть, что процент должников выше в группах, целью кредита которой является автомобиль или образование.

Данные показывают, что цель кредита влияет на его возврат в срок.

## Общие выводы

Мы посмотрели, как наличие детей, семейное положение, уровень дохода, а так же цели кредита влияют на возврат кредита в срок.

Если брать результаты исследования по группам, то:

В группе по наличию детей - самые надёжные клиенты это те, у которых нет детей (`7.5%` невозврата в срок); ненадёжные - клиенты с одним ребёнком (`9.2%` невозврата в срок) или более (`9.3%` невозврата в срок).

В группе по семейному положению - самые надёжные - клиенты с семейным положением вдовы / вдовца (`6.6%` невозврата в срок), а также те, кто в браке (`7.5%` невозврата в срок); ненадёжные - находящиеся в гражданском браке (`9.3%` невозврата в срок) и не находящиеся в браке (`9.8%` невозврата в срок).

В группе по доходу - самые надёжные клиенты это те, у кого доход выше среднего (`7.5%` невозврата в срок); ненадёжные - у кого средний доход (`8.7%` невозврата в срок).

В группе по цели кредита - самые надёжные клиенты это те, кто берут кредита для недвижимости (`7.2%` невозврата в срок); ненадёжные - кто берут кредит для образования (`9.2%` невозврата в срок) или автомобиля (`9.4%` невозврата в срок).